In [14]:
num='112'
i=0
window=num[i]+num[i+1]+num[i+2]
num[i]==num[i+1]==num[i+2]

False

In [12]:
(n & (n - 1)) == 0 and (n - 1) % 3 == 0

True

In [4]:
(0/4)%4==0

True

In [ ]:
def largestGoodInteger(num: str) -> str:
    result = ''
    for i in range(len(num) - 2):
        window = num[i:i+3]
        if window[0] == window[1] == window[2]:
            if result == '' or int(window) > int(result):
                result = window
    return result
        

In [23]:
class Solution:
    def isPowerOfThree(self, n: int) -> bool:
        if n <= 0:
            return False
        while n % 3 == 0:
            n //= 3
        return n == 1
                

'999'

In [27]:
import math

def is_power_of_three(n: int) -> bool:
    return n > 0 and 3 ** round(math.log(n, 3)) == n


2

In [1]:
import requests
url = "http://localhost:8000/add"  # Replace with your endpoint URL
data = {
    'a':100,
    'b':100
}
headers={
    'accept':'application/json',
    'Content-Type':'application/json'
}

response = requests.post(url, json=data,headers=headers)

In [3]:
response.text

'{"result":200.0}'

In [4]:
ip1={
    
            "type": "object",
            "properties": {
            "a": {"type": "integer", "description": "First integer to add"},
            "b": {"type": "integer", "description": "Second integer to add"}
            },
            "required": ["a", "b"]
        }

In [5]:
ip1==input_scheme[0]

True

In [1]:
input_scheme=[
    {
        "type": "object",
            "properties": {
            "a": {"type": "integer", "description": "First integer to add"},
            "b": {"type": "integer", "description": "Second integer to add"}
            },
            "required": ["a", "b"]
    },
    {
        "type": "object",
            "properties": {
            "a": {"type": "integer", "description": "First integer to Multiply"},
            "b": {"type": "float", "description": "Second integer to Multiply"}
            },
        
            "required": ["a", "b"]
        
    }
]

In [3]:
type(input_scheme[1])


dict

# Building Parser


In [8]:
import requests
import json
from mcp.types import Tool

def parse_openapi(openapi_url: str) -> list[Tool]:
    """Parse OpenAPI spec and return a list of MCP Tools"""
    response = requests.get(openapi_url)
    response.raise_for_status()
    spec = response.json()

    tools = []

    # Iterate over all paths
    for path, methods in spec.get("paths", {}).items():
        for method, details in methods.items():
            tool_name = details.get("operationId", f"{method}_{path}")
            description = details.get("description", details.get("summary", ""))

            # Collect input schema (body + query params)
            properties = {}
            required = []

            # Query params
            for param in details.get("parameters", []):
                pname = param["name"]
                ptype = param.get("schema", {}).get("type", "string")
                properties[pname] = {"type": ptype, "description": param.get("description", "")}
                if param.get("required"):
                    required.append(pname)

            # Request body schema
            if "requestBody" in details:
                content = details["requestBody"]["content"]
                if "application/json" in content:
                    schema = content["application/json"]["schema"]
                    if "properties" in schema:
                        properties.update(schema["properties"])
                        required.extend(schema.get("required", []))

            # Build MCP Tool
            tool = Tool(
                name=tool_name,
                inputSchema={
                    "type": "object",
                    "properties": properties,
                    "required": required
                }
            )
            tools.append(tool)

    return tools
tools=parse_openapi("http://localhost:8000/openapi.json")
print(tools)


[Tool(name='add_add_post', title=None, description=None, inputSchema={'type': 'object', 'properties': {}, 'required': []}, outputSchema=None, annotations=None, meta=None), Tool(name='multiply_multiply_post', title=None, description=None, inputSchema={'type': 'object', 'properties': {}, 'required': []}, outputSchema=None, annotations=None, meta=None), Tool(name='forecast_get_forecast_post', title=None, description=None, inputSchema={'type': 'object', 'properties': {'latitude': {'type': 'number', 'description': ''}, 'longitude': {'type': 'number', 'description': ''}}, 'required': ['latitude', 'longitude']}, outputSchema=None, annotations=None, meta=None), Tool(name='alerts_get_alerts_post', title=None, description=None, inputSchema={'type': 'object', 'properties': {}, 'required': []}, outputSchema=None, annotations=None, meta=None)]


In [15]:
j=tools[0].json()
j2=tools[1].json()
j3=tools[2].json()



/var/folders/zs/qf7bdgx10sg_h0m7bqz7r08w0000gn/T/ipykernel_12314/2050117476.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  j=tools[0].json()
/var/folders/zs/qf7bdgx10sg_h0m7bqz7r08w0000gn/T/ipykernel_12314/2050117476.py:2: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  j2=tools[1].json()
/var/folders/zs/qf7bdgx10sg_h0m7bqz7r08w0000gn/T/ipykernel_12314/2050117476.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  j3=tools[2].json()


In [16]:
j2

'{"name":"multiply_multiply_post","title":null,"description":null,"inputSchema":{"type":"object","properties":{},"required":[]},"outputSchema":null,"annotations":null,"meta":null}'

In [14]:
print(j)

{"name":"add_add_post","title":null,"description":null,"inputSchema":{"type":"object","properties":{},"required":[]},"outputSchema":null,"annotations":null,"meta":null}


In [17]:
import json

def resolve_ref(ref: str, spec: dict):
    """Resolve a $ref like '#/components/schemas/WeatherRequest'."""
    parts = ref.lstrip("#/").split("/")
    obj = spec
    for part in parts:
        obj = obj[part]
    return obj

def parse_openapi(openapi_json: dict):
    endpoints = []

    for path, methods in openapi_json.get("paths", {}).items():
        for method, details in methods.items():
            endpoint = {
                "path": path,
                "method": method.upper(),
                "summary": details.get("summary"),
                "parameters": [],
                "requestBody": None,
                "responses": {}
            }

            # Parameters (query/path)
            for param in details.get("parameters", []):
                schema = param.get("schema", {})
                if "$ref" in schema:
                    schema = resolve_ref(schema["$ref"], openapi_json)
                endpoint["parameters"].append({
                    "name": param["name"],
                    "in": param["in"],
                    "required": param.get("required", False),
                    "schema": schema
                })

            # Request Body
            request_body = details.get("requestBody")
            if request_body:
                content = request_body.get("content", {})
                if "application/json" in content:
                    schema = content["application/json"]["schema"]
                    if "$ref" in schema:
                        schema = resolve_ref(schema["$ref"], openapi_json)
                    endpoint["requestBody"] = {
                        "required": request_body.get("required", False),
                        "schema": schema
                    }

            # Responses
            for status, response in details.get("responses", {}).items():
                content = response.get("content", {})
                if "application/json" in content:
                    schema = content["application/json"].get("schema", {})
                    if "$ref" in schema:
                        schema = resolve_ref(schema["$ref"], openapi_json)
                    endpoint["responses"][status] = schema

            endpoints.append(endpoint)

    return endpoints


In [21]:
config={"openapi":"3.1.0","info":{"title":"FastAPI","version":"0.1.0"},"paths":{"/add":{"post":{"summary":"Add","operationId":"add_add_post","requestBody":{"content":{"application/json":{"schema":{"$ref":"#/components/schemas/Numbers"}}},"required":True},"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}}},"/multiply":{"post":{"summary":"Multiply","operationId":"multiply_multiply_post","requestBody":{"content":{"application/json":{"schema":{"$ref":"#/components/schemas/Numbers"}}},"required":True},"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}}},"/get_forecast":{"post":{"summary":"Forecast","operationId":"forecast_get_forecast_post","parameters":[{"name":"latitude","in":"query","required":True,"schema":{"type":"number","title":"Latitude"}},{"name":"longitude","in":"query","required":True,"schema":{"type":"number","title":"Longitude"}}],"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}}},"/get_alerts":{"post":{"summary":"Alerts","operationId":"alerts_get_alerts_post","requestBody":{"content":{"application/json":{"schema":{"$ref":"#/components/schemas/WeatherRequest"}}},"required":True},"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}}}},"components":{"schemas":{"HTTPValidationError":{"properties":{"detail":{"items":{"$ref":"#/components/schemas/ValidationError"},"type":"array","title":"Detail"}},"type":"object","title":"HTTPValidationError"},"Numbers":{"properties":{"a":{"type":"number","title":"A"},"b":{"type":"number","title":"B"}},"type":"object","required":["a","b"],"title":"Numbers"},"ValidationError":{"properties":{"loc":{"items":{"anyOf":[{"type":"string"},{"type":"integer"}]},"type":"array","title":"Location"},"msg":{"type":"string","title":"Message"},"type":{"type":"string","title":"Error Type"}},"type":"object","required":["loc","msg","type"],"title":"ValidationError"},"WeatherRequest":{"properties":{"state":{"type":"string","title":"State"}},"type":"object","required":["state"],"title":"WeatherRequest"}}}}


In [24]:
spec =config

endpoints = parse_openapi(spec)

for ep in endpoints:
    print(ep["method"], ep["path"])
    if ep["requestBody"]:
        print("  Request Schema:", ep["requestBody"]["schema"])
    if ep["parameters"]:
        print("  Parameters:", ep["parameters"])


POST /add
  Request Schema: {'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'type': 'object', 'required': ['a', 'b'], 'title': 'Numbers'}
POST /multiply
  Request Schema: {'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'type': 'object', 'required': ['a', 'b'], 'title': 'Numbers'}
POST /get_forecast
  Parameters: [{'name': 'latitude', 'in': 'query', 'required': True, 'schema': {'type': 'number', 'title': 'Latitude'}}, {'name': 'longitude', 'in': 'query', 'required': True, 'schema': {'type': 'number', 'title': 'Longitude'}}]
POST /get_alerts
  Request Schema: {'properties': {'state': {'type': 'string', 'title': 'State'}}, 'type': 'object', 'required': ['state'], 'title': 'WeatherRequest'}


In [25]:
import json
import requests
import logging as logger
from copy import deepcopy
from typing import List
from mcp.types import Tool, TextContent

# --------------------------
# OpenAPI Parser
# --------------------------

def resolve_ref(ref: str, spec: dict):
    """Resolve a $ref like '#/components/schemas/WeatherRequest'."""
    parts = ref.lstrip("#/").split("/")
    obj = spec
    for part in parts:
        obj = obj[part]
    return deepcopy(obj)

def expand_schema(schema: dict, spec: dict):
    """Recursively expand schema by resolving $ref."""
    if not isinstance(schema, dict):
        return schema

    if "$ref" in schema:
        schema = resolve_ref(schema["$ref"], spec)

    if "properties" in schema:
        for k, v in schema["properties"].items():
            schema["properties"][k] = expand_schema(v, spec)

    if "items" in schema:
        schema["items"] = expand_schema(schema["items"], spec)

    return schema

def parse_openapi_to_tools(openapi_json: dict):
    """
    Parse OpenAPI spec and return:
      1. tools (list of Tool objects)
      2. tool_map (metadata for call_tool)
    """
    tools = []
    tool_map = {}

    for path, methods in openapi_json.get("paths", {}).items():
        for method, details in methods.items():
            tool_name = details.get("summary", path).replace(" ", "_")

            # Default input schema
            input_schema = {"type": "object", "properties": {}, "required": []}

            # Parameters (query/path)
            for param in details.get("parameters", []):
                schema = expand_schema(param.get("schema", {}), openapi_json)
                input_schema["properties"][param["name"]] = schema
                if param.get("required", False):
                    input_schema.setdefault("required", []).append(param["name"])

            # Request Body
            request_body = details.get("requestBody")
            if request_body:
                content = request_body.get("content", {})
                if "application/json" in content:
                    schema = expand_schema(content["application/json"]["schema"], openapi_json)
                    input_schema = schema  # override with request schema

            # Register Tool
            tool = Tool(
                name=tool_name,
                inputSchema=input_schema
            )
            tools.append(tool)

            # Store for call_tool lookup
            tool_map[tool_name] = {
                "path": path,
                "method": method.upper(),
                "headers": {
                    "accept": "application/json",
                    "Content-Type": "application/json"
                }
            }

    return tools, tool_map

In [26]:
tools, tool_map = parse_openapi_to_tools(spec)

In [28]:
tool_map

{'Add': {'path': '/add',
  'method': 'POST',
  'headers': {'accept': 'application/json',
   'Content-Type': 'application/json'}},
 'Multiply': {'path': '/multiply',
  'method': 'POST',
  'headers': {'accept': 'application/json',
   'Content-Type': 'application/json'}},
 'Forecast': {'path': '/get_forecast',
  'method': 'POST',
  'headers': {'accept': 'application/json',
   'Content-Type': 'application/json'}},
 'Alerts': {'path': '/get_alerts',
  'method': 'POST',
  'headers': {'accept': 'application/json',
   'Content-Type': 'application/json'}}}

In [27]:
tools

[Tool(name='Add', title=None, description=None, inputSchema={'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'type': 'object', 'required': ['a', 'b'], 'title': 'Numbers'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='Multiply', title=None, description=None, inputSchema={'properties': {'a': {'type': 'number', 'title': 'A'}, 'b': {'type': 'number', 'title': 'B'}}, 'type': 'object', 'required': ['a', 'b'], 'title': 'Numbers'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='Forecast', title=None, description=None, inputSchema={'type': 'object', 'properties': {'latitude': {'type': 'number', 'title': 'Latitude'}, 'longitude': {'type': 'number', 'title': 'Longitude'}}, 'required': ['latitude', 'longitude']}, outputSchema=None, annotations=None, meta=None),
 Tool(name='Alerts', title=None, description=None, inputSchema={'properties': {'state': {'type': 'string', 'title': 'State'}}, 'type': 'object', 'required': ['state

# Self

In [29]:
config

{'openapi': '3.1.0',
 'info': {'title': 'FastAPI', 'version': '0.1.0'},
 'paths': {'/add': {'post': {'summary': 'Add',
    'operationId': 'add_add_post',
    'requestBody': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/Numbers'}}},
     'required': True},
    'responses': {'200': {'description': 'Successful Response',
      'content': {'application/json': {'schema': {}}}},
     '422': {'description': 'Validation Error',
      'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'}}}}}}},
  '/multiply': {'post': {'summary': 'Multiply',
    'operationId': 'multiply_multiply_post',
    'requestBody': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/Numbers'}}},
     'required': True},
    'responses': {'200': {'description': 'Successful Response',
      'content': {'application/json': {'schema': {}}}},
     '422': {'description': 'Validation Error',
      'content': {'application/json': {'schema

In [37]:
config['paths']['/add']

{'post': {'summary': 'Add',
  'operationId': 'add_add_post',
  'requestBody': {'content': {'application/json': {'schema': {'$ref': '#/components/schemas/Numbers'}}},
   'required': True},
  'responses': {'200': {'description': 'Successful Response',
    'content': {'application/json': {'schema': {}}}},
   '422': {'description': 'Validation Error',
    'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'}}}}}}}